<a href="https://colab.research.google.com/github/ctruciosm/ME715-unicamp/blob/master/Lista07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install wooldridge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 23.2 MB/s eta 0:00:00


In [ ]:
import wooldridge as woo
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np

barium = woo.dataWoo('barium')
modelo = smf.ols(formula  = "np.log(chnimp) ~np.log(chempi) + np.log(gas) + np.log(rtwex) + befile6 + affile6 + afdec6", data = barium).fit()

u = np.log(barium.chnimp) - modelo.fittedvalues

u_lag1 = np.roll(u, shift = 1)
#u_lag1[0] = 0

reg_aux = sm.OLS(u, u_lag1).fit()
rho = reg_aux.params[0]

n = len(barium)
psi = np.identity(n)
psi[0,0] = np.sqrt(1-rho**2)

for i in range(1,n):
  psi[i,i] = 1
  psi[i, i-1] = -rho

Y = np.dot(psi, np.log(barium['chnimp']))
#X = np.dot(psi, barium[['chempi', 'gas', 'rtwex', 'befile6', 'affile6', 'afdec6']])

X = pd.DataFrame({'intercept' : [1]*n,
                  'lchempi' : np.log(barium['chempi']),
                 'lgas' : np.log(barium["gas"]),
                 'lrtwex': np.log(barium.rtwex),
                 'befile6' : barium.befile6,
                 'affile' : barium.affile6,
                 'afdec' : barium.afdec6})
X = np.dot(psi, X)

#new_df = pd.DataFrame(X, columns = ['chempi', 'gas', 'rtwex', 'befile6', 'affile6', 'afdec6'])
#new_df['lchnimp'] = Y

modelo_mqgf  = sm.OLS(Y, X).fit()
print(modelo_mqgf.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.985
Model:                            OLS   Adj. R-squared (uncentered):              0.984
Method:                 Least Squares   F-statistic:                              1165.
Date:                Thu, 26 Oct 2023   Prob (F-statistic):                   6.98e-110
Time:                        14:10:22   Log-Likelihood:                         -109.44
No. Observations:                 131   AIC:                                      232.9
Df Residuals:                     124   BIC:                                      253.0
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [ ]:
psi

array([[ 0.96267155,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.27067229,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        , -0.27067229,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ..., -0.27067229,
         1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.27067229,  1.        ]])

In [ ]:
modelo.